In [1]:
#准备数据
import numpy as np
print('preparing data...')
from sklearn import datasets
import numpy as np
iris=datasets.load_iris()
iris.keys()
X=iris['data'][:120]
y=(iris['target']).astype(np.int)
print('done!')
X=np.matrix(X).T
y=np.matrix(y).T
X.shape

preparing data...
done!


(4, 120)

In [2]:
class node(object):
    def __init__(self, dataset, depth=0, parent=None, median=None):
        self.parent = parent
        self.median = median
        self.depth = depth
        self.data = dataset
        self.create_child_node()

    def create_child_node(self):
        if self.data.shape[1] <= 1:
            if self.data.shape[1] == 1:
                self.value = self.data
            else:
                self.value = None
            return
        else:
            self.median = np.median(self.data[(self.depth)], axis=1)
            median = self.median
            left = self.data[np.repeat(self.data[self.depth, :] < median, self.data.shape[0]).reshape(
                self.data.shape[1], self.data.shape[0]).T]
            left = left.reshape(self.data.shape[0], int(
                left.shape[1] / self.data.shape[0]))
            right = self.data[np.repeat(self.data[self.depth, :] >= median, self.data.shape[0]).reshape(
                self.data.shape[1], self.data.shape[0]).T]
            right = right.reshape(self.data.shape[0], int(
                right.shape[1] / self.data.shape[0]))
            self.left = node(depth=(self.depth + 1) %
                             self.data.shape[0], parent=self, dataset=left)
            self.right = node(depth=(self.depth + 1) %
                              self.data.shape[0], parent=self, dataset=right)
            self.value = self.data[:, 0]


class kdd_KNN(object):
    def fit(self, X, y):
        self.X=X
        self.kdtree = node(self.X)

    def predict(self, x):
        x = x.T
        result = []
        i = 0
        for xi in x:
            i += 1
            xi = xi.T
            cloest = self.find(xi, self.kdtree)
            result.append(
                y[np.where((self.X == cloest).sum(axis=0) == xi.shape[0])[1]][0])
        return result

    def find(self, x, kdtree,depth=0):
        root = kdtree
        locate = kdtree
        cloest = locate.value
        side = None
        while locate.data.shape[1] > 1:
            if x[depth % x.shape[0]] < locate.median:
                locate = locate.left
                side = 'l'
            else:
                locate = locate.right
                side = 'r'
            depth+=1
            while locate.data.shape[1] == 0:
                locate = locate.parent
                if side == 'r':
                    locate = locate.left
                else:
                    locate = locate.right
            if (np.array(locate.value - x)**2).sum() < (np.array(cloest - x)**2).sum() and locate.value.shape[1] != 0:
                cloest = locate.value
        cloest = locate.data
        while(locate != root):
            locate = locate.parent
            depth-=1
            if side == 'r':
                other = self.find(x, locate.left,depth+1)
            else:
                other = self.find(x, locate.right,depth+1)

            if (np.array(other - x)**2).sum() < (np.array(cloest - x)**2).sum() and other.shape[1] != 0:
                cloest = other
        return cloest

In [3]:
clf=kdd_KNN()
clf.fit(X,y)
clf.predict(X)

AttributeError: 'numpy.ndarray' object has no attribute '_collapse'